In [6]:
import pandas
from reactome2py import analysis, content, fiviz
from collections import defaultdict
from itertools import chain
from operator import methodcaller

#### Assessing pathways and drug targets in Cyclic Multiplexed-Immunofluorescence (cmIF) image biomarkers
Resource: https://www.ncbi.nlm.nih.gov/pubmed/31502168

1. After mapping proteins to gene names, we wrangle the matrix into a dictionary where each image section would hold the biomarkers as its values. 

In [3]:
path = "data/marker_matrix_gene.csv"

df = pandas.read_csv(path, index_col=0)
df = df.apply(lambda x : x.str.split(","), axis=0)
row_dict = df.T.to_dict('records')

def imggenes2dict(row_dict):
    dd = defaultdict(list)
    dict_items = map(methodcaller('items'), row_dict)
    
    for k, v in chain.from_iterable(dict_items):
        dd[k].extend(v)
    
    img_dict = dict(dd)
    return(img_dict)

img_dict = imggenes2dict(row_dict)

2. Now we can pass the biomarkers to fetch all possible reactome pathways available for each image segment. 

In [8]:
print(img_dict)

{'R1': ['MS4A1', 'CD8A', 'CD4', 'KRT19'], 'R2': ['PCNA', 'CD68', 'PDCD1', 'PTRPC'], 'R3': ['KRT8', 'HER2', 'FOXP3', 'KRT5'], 'R4': ['H3F3A', 'H3F3B', 'RPS6', 'ESR1', 'CD44'], 'R5': ['KRT17', 'PDPN', 'PECAM1', 'GZMB'], 'R6': ['VIM', 'pAb', 'RB1', 'CD3'], 'R7': ['ACTA2', 'PARP1', 'H2AFX', 'CDH1'], 'R8': ['KRT7', 'KRT14', 'COL4A1', 'LMNA'], 'R9': ['H3K27', 'CD274', 'MKI67', 'PGR'], 'R10': ['LMNB1', 'H3K4', 'LMNB2', 'COL1A1'], 'R11': ['CD34', 'AR', 'HIF1A', 'FOXP3'], 'R12': ['AKT1', 'BMP2', 'BMP4', 'MAPK1 ', 'MAPK3'], 'R13': ['BRD4', 'CASP3', 'NCAM1', 'MTOR']}


In [4]:
result = [analysis.identifiers(ids=",".join(d)) for d in img_dict.values()]
tokens = [r['summary']['token'] for r in result]


def get_pathways(t):
    token_result = analysis.token(token=t, species='Homo sapiens', page_size='-1', page='-1', sort_by='ENTITIES_FDR', 
                                  order='ASC', resource='TOTAL', p_value='1', include_disease=True, 
                                  min_entities=None, max_entities=None)
    l = [p['name'] for p in token_result['pathways']]
    l.sort() 
    return(l)

pathways = [get_pathways(t) for t in tokens]

3. Given the biomarkers, we can also fetch all available drug targets from Drug Central.

In [9]:
print(pathways)

[['Adaptive Immune System', 'Alpha-defensins', 'Antimicrobial peptides', 'Binding and entry of HIV virion', 'Cargo recognition for clathrin-mediated endocytosis', 'Clathrin-mediated endocytosis', 'Costimulation by the CD28 family', 'Cytokine Signaling in Immune system', 'Defensins', 'Developmental Biology', 'Disease', 'Downstream TCR signaling', 'Early Phase of HIV Life Cycle', 'Formation of the cornified envelope', 'Generation of second messenger molecules', 'HIV Infection', 'HIV Life Cycle', 'Host Interactions of HIV factors', 'Immune System', 'Immunoregulatory interactions between a Lymphoid and a non-Lymphoid cell', 'Infectious disease', 'Innate Immune System', 'Keratinization', 'Membrane Trafficking', 'Nef Mediated CD4 Down-regulation', 'Nef-mediates down modulation of cell surface receptors by recruiting them to clathrin adapters', 'Other interleukin signaling', 'PD-1 signaling', 'Phosphorylation of CD3 and TCR zeta chains', 'Signaling by Interleukins', 'TCR signaling', 'The role

In [10]:
result_drug_targets = [fiviz.genelist_drug_target(ids=",".join(d), source="drugcentral") for d in img_dict.values()]
print(result_drug_targets)

[{'interaction': [{'interactionID': '13173', 'drug': '4984', 'target': '1508', 'interactionType': 'ANTIBODY BINDING', 'source': '1331', 'expEvidence': '16425', 'id': '13173'}, {'interactionID': '13404', 'drug': '5223', 'target': '1508', 'interactionType': 'ANTIBODY BINDING', 'source': '1331', 'expEvidence': '16698', 'id': '13404'}, {'interactionID': '13122', 'drug': '4951', 'target': '1508', 'interactionType': 'ANTIBODY BINDING', 'source': '1331', 'expEvidence': '16385', 'id': '13122'}, {'interactionID': '13172', 'drug': '4983', 'target': '1508', 'interactionType': 'ANTIBODY BINDING', 'source': '1331', 'expEvidence': '16424', 'id': '13172'}, {'interactionID': '13163', 'drug': '4975', 'target': '1508', 'interactionType': 'ANTIBODY BINDING', 'source': '1331', 'expEvidence': '16415', 'id': '13163'}], 'drug': [{'drugID': '4975', 'drugName': 'Rituximab', 'id': '4975'}, {'drugID': '4984', 'drugName': 'Ibritumomab tiuxetan', 'id': '4984'}, {'drugID': '4951', 'drugName': 'Ofatumumab', 'id': '4